In [4]:

import os
import keras
import json
import glob
import sys
import pandas as pd
import tensorflow as tf
from keras.layers import Input
import numpy as np
import argparse
#from keras_applications.resnext import ResNeXt50
from keras.utils.data_utils import get_file
#import face_recognition
import matplotlib.pyplot as plt
import seaborn as sns

from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,BatchNormalization
from tensorflow.keras.layers import Dense, MaxPooling2D,Conv2D
from tensorflow.keras.layers import Input,Activation,Add,Normalization
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
! pip install keras_tuner
import keras_tuner as kt
from keras import backend as K
#sys.path.append('/content/drive/MyDrive/PhD/CD_ML/CD_ML/alciML')
import fnmatch
import re
import copy
import matplotlib.pyplot as plt
import scipy.signal
! pip install mpu
import mpu
#from script_onelist_colab import norm_int,repair,csv_exp,sel_exp,maxima,import_raw,read_processed,read_processed_sel,correct,onelist,interpolate_mat,interpolation_spectra

def touch(fname):
    if os.path.exists(fname):
        os.utime(fname, None)
    else:
        os.mkdir(fname)

#folder='/content/drive/MyDrive/PhD/CD_ML/CD_ML/alciML/csv/'
#if not os.path.abspath('')=='/content/drive/MyDrive/PhD/CD_ML/CD_ML/alciML/csv':
#  os.chdir(folder)
#print(os.path.abspath(''))  
#touch(folder+'png')





In [31]:
datadict={}

In [32]:
def noise(source):
  noisy=copy.deepcopy(source)
  smax=np.amax(noisy[:,1:])
  for j in range(len(noisy[0,:])):
    for i in range(len(noisy[:,0])):
      if j>0:
        noisy[i,j]+=np.random.uniform(0,2)
        noisy[i,j]+=np.random.uniform(0,1)*0.01*noisy[i,j]
  print(np.amax(noisy[:,1:]))
  noisy[:,1:]=noisy[:,1:]/np.amax(noisy[:,1:])*smax
  noisy[:,1:][noisy[:,1:] < 0] = 0
  noisy[:,1:]=np.rint(noisy[:,1:])
  #print(np.shape(noisy))
  return noisy


In [33]:
files=glob.glob("csv/png/sel_obs2nm_blank_3000uleth*.csv")

In [34]:
    fileslist=dict.fromkeys(files,1)
    i=1
    for key in fileslist:
        td=pd.read_csv(key,index_col=0)
        # col[key[4:-4]+'.txt']=td.columns
        tdn=np.array(td)
        datadict[key[8:-4]+'.txt']=tdn.astype(np.float)
        i+=1

In [35]:
#df3_copy=copy.deepcopy(df3)

def add_blank(x):
  if 'blank' not in x.file_name:
    ret=df2.loc[(df2['CD_amount'] == 'blank') & (df2['typ'] == x['typ']),'spec'].iloc[0]
    print(ret)
  else:
    ret=np.nan
  return noise(ret)

In [36]:
def fill(x,source):
  y=x
  for col in source.columns:

    if np.isnan(np.amax(np.amax(x[col]))):
      nancheck=0
      while nancheck==0:
        newval=source[col].iloc[np.random.randint(0,len(source))]

        if not np.isnan(np.amax(np.amax(newval))):
          print('yes')
          nancheck=1
     # print(np.shape(noise(newval)))
      y[col]=noise(newval)    
  return y

In [37]:
df3=pd.read_pickle('csv/PL_LT_sol_aug_pd.pkl')

In [38]:
checker3='2nm_CD032_0.5_(.*)ul_(.*)ulethanol_(.*)ulwater_(.*)_(.*).txt'
checker='sel_obs2nm_blank_3000uleth(.*)_(.*)_(.*)'
df2=pd.DataFrame(index=None,columns=('alcohol_concentration','CD_amount','typ','file_name','spec'))
for key in datadict: 
  if re.search(checker,key) is not None: 
    con='blank'
    am='blank'
    typ=re.search(checker,key).group(2)
    dat=pd.DataFrame([[con,am,typ,key,datadict[key]]]) 
    dat.columns=df3.columns 
    #df3=df3.append(dat,ignore_index=True) 
    df2=df2.append(dat,ignore_index=True) 

In [39]:
df3=df3.drop(df3.loc[df3['typ']=='LT'].index)

In [40]:
df3['blank_spec']=np.nan
for typ in df3['typ'].unique():
  print(typ)
  df3['blank_spec'] = df3.apply(lambda x: add_blank(x) if x['typ']==str(typ) else x['blank_spec'], axis=1)


PL
[[ 300. 2058. 2011. ...    0.    0.    0.]
 [ 320.    0.    0. ...    0.    0.    0.]
 [ 340.    0.    0. ...    0.    0.    0.]
 ...
 [ 560.    0.    0. ...   71.    0.    0.]
 [ 580.    0.    0. ...   62.   43.   62.]
 [ 600.    0.    0. ...   33.  107.   70.]]
3006.1402991147606
[[ 300. 2058. 2011. ...    0.    0.    0.]
 [ 320.    0.    0. ...    0.    0.    0.]
 [ 340.    0.    0. ...    0.    0.    0.]
 ...
 [ 560.    0.    0. ...   71.    0.    0.]
 [ 580.    0.    0. ...   62.   43.   62.]
 [ 600.    0.    0. ...   33.  107.   70.]]
3027.3158811522726
[[ 300. 2058. 2011. ...    0.    0.    0.]
 [ 320.    0.    0. ...    0.    0.    0.]
 [ 340.    0.    0. ...    0.    0.    0.]
 ...
 [ 560.    0.    0. ...   71.    0.    0.]
 [ 580.    0.    0. ...   62.   43.   62.]
 [ 600.    0.    0. ...   33.  107.   70.]]
3013.3204166830815
[[ 300. 2058. 2011. ...    0.    0.    0.]
 [ 320.    0.    0. ...    0.    0.    0.]
 [ 340.    0.    0. ...    0.    0.    0.]
 ...
 [ 560.    0. 

In [41]:
df3=df3.drop(df3.loc[df3['CD_amount']=='blank'].index)

In [42]:
df3['blank_LT405spec'] = df3.apply(lambda x: x['blank_spec'] if x['typ']=='LT405' else np.nan, axis=1)
df3['blank_LT450spec'] = df3.apply(lambda x: x['blank_spec'] if x['typ']=='LT450' else np.nan, axis=1)
df3['blank_PLspec'] = df3.apply(lambda x: x['blank_spec'] if x['typ']=='PL' else np.nan, axis=1)

In [43]:
df3['LT405spec'] = df3.apply(lambda x: x['spec'] if x['typ']=='LT405' else np.nan, axis=1)
df3['LT450spec'] = df3.apply(lambda x: x['spec'] if x['typ']=='LT450' else np.nan, axis=1)
df3['PLspec'] = df3.apply(lambda x: x['spec'] if x['typ']=='PL' else np.nan, axis=1)


In [44]:
df3=df3.drop('blank_spec',axis=1)

In [45]:
for amount in df3['CD_amount'].unique():
  print(amount)
  df3[str(int(amount))+'_LT405spec'] = df3.apply(lambda x: x['LT405spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  df3[str(int(amount))+'_LT450spec'] = df3.apply(lambda x: x['LT450spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  df3[str(int(amount))+'_PLspec'] = df3.apply(lambda x: x['PLspec'] if x['CD_amount']==int(amount) else np.nan, axis=1)

300.0
100.0
150.0


In [46]:
df3=df3.drop('LT405spec',axis=1)
df3=df3.drop('LT450spec',axis=1)
df3=df3.drop('PLspec',axis=1)

In [47]:
df3=df3.drop('spec',axis=1)

In [48]:
cl=df3.iloc[:,4:].columns
c=df3.iloc[:,7:].columns

In [49]:
df4=pd.DataFrame(columns=df3.columns)

In [50]:

for alccon in df3['alcohol_concentration'].unique():
  print(alccon)
  df_tmp=copy.deepcopy(df3.loc[df3['alcohol_concentration']==alccon])
  df_tmp= df_tmp.apply(lambda x: pd.Series(x.dropna().values))
  no=max(df_tmp[c].count())
  df_tmp=df_tmp.iloc[:no,:]
  df_tmp=df_tmp.apply(lambda x: fill(x,df_tmp[cl]) ,axis=1)
  df4=pd.concat((df4,df_tmp))
  del df_tmp
  #df3[str(int(amount))+'_LT405spec'] = df3.apply(lambda x: x['LT405spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  #df3[str(int(amount))+'_LT450spec'] = df3.apply(lambda x: x['LT450spec'] if x['CD_amount']==int(amount) else np.nan, axis=1)
  #df3[str(int(amount))+'_PLspec'] = df3.apply(lambda x: x['PLspec'] if x['CD_amount']==int(amount) else np.nan, axis=1)

0.8
yes
64361
yes
65172
yes
12568
yes
65099
yes
64755
yes
28824
yes
64305
yes
65143
yes
12495
yes
65209
yes
65064
yes
65364
yes
28776
yes
64621
yes
64868
yes
18897
yes
65357
yes
65339
yes
64906
yes
28894
yes
65116
yes
65154
yes
18853
yes
65016
yes
65044
yes
65166
yes
28872
yes
65458
yes
64954
yes
12536
yes
63872
yes
64748
yes
65069
yes
44940
1.0
yes
65500
yes
64308
yes
6762
yes
65019
yes
65437
yes
64574
yes
65515
yes
19548
0.4666666666666667
yes
64735
yes
65094
yes
65077
yes
64451
yes
15507
0.06666666666666667
yes
65296
yes
65160
yes
4480
yes
63706
yes
65261
yes
8165
yes
65290
yes
65098
yes
4494
yes
63340
yes
64274
yes
63629
yes
62246
yes
8187
0.6333333333333333
yes
65036
yes
64998
yes
6926
yes
65395
yes
64888
yes
15754
0.9666666666666667
yes
65434
yes
65237
yes
64570
yes
7224
yes
64139
yes
65121
yes
18572
yes
64683
yes
65464
yes
7224
yes
65367
yes
65153
yes
65507
yes
64728
yes
18566
yes
65400
yes
64427
yes
8141
yes
64164
yes
65383
yes
65428
yes
65337
yes
18890
0.2
yes
64932
yes
65356


In [51]:
df4=df4.reset_index(drop=True)
df4=df4.drop(['CD_amount','typ','file_name'],axis=1)

In [52]:
df4.to_pickle('PL_LT_augmented_onemat_pd4.pkl')

In [53]:
df4=pd.read_pickle('PL_LT_augmented_onemat_pd4.pkl')

In [54]:
df4

,alcohol_concentration,blank_LT405spec,blank_LT450spec,blank_PLspec,300_LT405spec,300_LT450spec,300_PLspec,100_LT405spec,100_LT450spec,100_PLspec,150_LT405spec,150_LT450spec,150_PLspec
0,0.800000,"[[0.0, 64.0], [1.0, 65.0], [2.0, 59.0], [3.0, ...","[[0.0, 33.0], [1.0, 31.0], [2.0, 44.0], [3.0, ...","[[300.0, 2066.0, 2021.0, 2150.0, 2301.0, 2647....","[[0, 58], [1, 58], [2, 63], [3, 68], [4, 62], ...","[[0, 55], [1, 55], [2, 61], [3, 74], [4, 66], ...","[[300, 0, 0, 7, 0, 3, 0, 0, 4, 1, 0, 11, 0, 6,...","[[0, 74], [1, 74], [2, 65], [3, 57], [4, 58], ...","[[0, 63], [1, 63], [2, 68], [3, 74], [4, 64], ...","[[300, 0, 4, 6, 5, 0, 14, 0, 10, 2, 16, 2, 0, ...","[[0, 64], [1, 64], [2, 64], [3, 53], [4, 54], ...","[[0, 62], [1, 62], [2, 43], [3, 69], [4, 66], ...","[[300, 4, 0, 0, 14, 7, 7, 13, 10, 3, 8, 12, 16..."
1,0.800000,"[[0.0, 65.0], [1.0, 65.0], [2.0, 57.0], [3.0, ...","[[0.0, 33.0], [1.0, 32.0], [2.0, 44.0], [3.0, ...","[[300.0, 2061.0, 2018.0, 2150.0, 2285.0, 2654....","[[0, 43], [1, 84], [2, 36], [3, 109], [4, 59],...","[[0, 24], [1, 55], [2, 29], [3, 90], [4, 117],...","[[300, 0, 0, 0, 1, 2, 10, 6, 3, 0, 8, 4, 4, 0,...","[[0, 99], [1, 99], [2, 111], [3, 99], [4, 98],...","[[0, 88], [1, 88], [2, 99], [3, 99], [4, 92], ...","[[300, 1, 0, 0, 4, 9, 0, 5, 20, 21, 14, 24, 23...","[[0, 88], [1, 23], [2, 70], [3, 90], [4, 59], ...","[[0, 35], [1, 13], [2, 13], [3, 80], [4, 55], ...","[[300, 8, 0, 0, 2, 0, 1, 5, 5, 1, 1, 1, 4, 0, ..."
2,0.800000,"[[0.0, 65.0], [1.0, 64.0], [2.0, 59.0], [3.0, ...","[[0.0, 32.0], [1.0, 33.0], [2.0, 43.0], [3.0, ...","[[300.0, 2063.0, 2012.0, 2134.0, 2285.0, 2633....","[[0, 44], [1, 85], [2, 36], [3, 112], [4, 61],...","[[0, 55], [1, 56], [2, 62], [3, 74], [4, 67], ...","[[300, 0, 0, 0, 1, 1, 10, 5, 3, 0, 7, 3, 3, 0,...","[[0, 61], [1, 75], [2, 70], [3, 104], [4, 10],...","[[0, 52], [1, 52], [2, 58], [3, 63], [4, 47], ...","[[300, 2, 0, 0, 7, 15, 6, 18, 14, 9, 11, 20, 1...","[[0, 65], [1, 65], [2, 65], [3, 53], [4, 54], ...","[[0, 35], [1, 14], [2, 13], [3, 80], [4, 55], ...","[[300, 8, 0, 0, 1, 0, 0, 5, 4, 1, 0, 1, 3, 0, ..."
3,0.800000,"[[0.0, 65.0], [1.0, 65.0], [2.0, 59.0], [3.0, ...","[[0.0, 33.0], [1.0, 33.0], [2.0, 44.0], [3.0, ...","[[300.0, 2058.0, 2007.0, 2130.0, 2274.0, 2641....","[[0, 44], [1, 85], [2, 36], [3, 111], [4, 60],...","[[0, 24], [1, 55], [2, 30], [3, 91], [4, 117],...","[[300, 0, 0, 0, 0, 2, 9, 6, 3, 0, 7, 4, 4, 0, ...","[[0, 75], [1, 74], [2, 66], [3, 58], [4, 58], ...","[[0, 104], [1, 26], [2, 57], [3, 102], [4, 85]...","[[300, 0, 0, 0, 0, 4, 0, 0, 2, 10, 6, 4, 7, 18...","[[0, 64], [1, 65], [2, 64], [3, 53], [4, 54], ...","[[0, 35], [1, 14], [2, 14], [3, 81], [4, 56], ...","[[300, 7, 0, 0, 1, 0, 1, 4, 4, 0, 0, 1, 3, 0, ..."
4,0.800000,"[[0.0, 65.0], [1.0, 65.0], [2.0, 57.0], [3.0, ...","[[0.0, 32.0], [1.0, 31.0], [2.0, 44.0], [3.0, ...","[[300.0, 2054.0, 2017.0, 2128.0, 2299.0, 2651....","[[0, 44], [1, 86], [2, 37], [3, 111], [4, 60],...","[[0, 25], [1, 55], [2, 29], [3, 91], [4, 118],...","[[300, 0, 0, 7, 0, 3, 0, 0, 3, 0, 0, 11, 0, 5,...","[[0, 74], [1, 74], [2, 65], [3, 58], [4, 59], ...","[[0, 22], [1, 47], [2, 75], [3, 75], [4, 67], ...","[[300, 5, 0, 0, 4, 4, 0, 8, 0, 6, 4, 4, 2, 15,...","[[0, 65], [1, 65], [2, 64], [3, 53], [4, 55], ...","[[0, 35], [1, 14], [2, 13], [3, 80], [4, 56], ...","[[300, 8, 0, 0, 2, 0, 0, 4, 4, 0, 0, 0, 4, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,0.320000,"[[0.0, 65.0], [1.0, 65.0], [2.0, 57.0], [3.0, ...","[[0.0, 32.0], [1.0, 32.0], [2.0, 44.0], [3.0, ...","[[300.0, 2073.0, 2025.0, 2149.0, 2296.0, 2662....","[[0, 80], [1, 53], [2, 70], [3, 48], [4, 56], ...","[[0, 93], [1, 39], [2, 41], [3, 115], [4, 3], ...","[[300, 5, 0, 0, 8, 0, 0, 8, 3, 0, 3, 0, 4, 0, ...","[[0, 56], [1, 85], [2, 102], [3, 90], [4, 132]...","[[0, 70], [1, 123], [2, 121], [3, 96], [4, 82]...","[[300, 22, 10, 3, 27, 8, 23, 16, 31, 11, 13, 1...","[[0, 69], [1, 45], [2, 0], [3, 106], [4, 51], ...","[[0, 109], [1, 35], [2, 73], [3, 32], [4, 34],..."